<a href="https://colab.research.google.com/github/KK-marius/Project-NLP-bag-of-words/blob/main/Text_classification_from_bag_of_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Classification

# Preliminaries

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

## Dataset



## TO DO: Take a look at the data

In [ ]:
from sklearn.datasets import fetch_20newsgroups
twenty = fetch_20newsgroups(subset='all',
    categories=['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med'],
    shuffle=True, random_state=42)

In [ ]:
twenty.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [ ]:
len(twenty.data)

3759

In [ ]:
print(twenty.data[0])

From: geb@cs.pitt.edu (Gordon Banks)
Subject: Re: "CAN'T BREATHE"
Article-I.D.: pitt.19440
Reply-To: geb@cs.pitt.edu (Gordon Banks)
Organization: Univ. of Pittsburgh Computer Science
Lines: 23

In article <1993Mar29.204003.26952@tijc02.uucp> pjs269@tijc02.uucp (Paul Schmidt) writes:
>I think it is important to verify all procedures with proper studies to
>show their worthiness and risk.  I just read an interesting tidbit that 
>80% of the medical treatments are unproven and not based on scientific 
>fact.  For example, many treatments of prostate cancer are unproven and
>the treatment may be more dangerous than the disease (according to the
>article I read.)

Where did you read this?  I don't think this is true.  I think most
medical treatments are based on science, although it is difficult
to prove anything with certitude.  It is true that there are some
things that have just been found "to work", but we have no good
explanation for why.  But almost everything does have a scientific
r

In [ ]:
print(twenty.target_names[twenty.target[0]])

sci.med


In [ ]:
twenty.target[:10] # Affiche les 10 premiers labels (catégories) associés aux documents de l’ensemble

array([2, 3, 3, 1, 2, 3, 2, 1, 1, 1])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(twenty.data, twenty.target, test_size=0.2, random_state=42)

# Complete process

## TO DO: Correct the errors

## Step1: extract features from text files


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
X_test_counts = count_vect.transform(X_test)
print(X_train_counts.shape)

n_token = X_train_counts.shape[1]
tokens = {i: w for w, i in count_vect.vocabulary_.items()}
tokens = [tokens[i] for i in range(n_token)]
print(tokens[n_token//2:n_token//2+20])

# for printing perposes
X_train_as_pd = pd.DataFrame(X_train_counts.toarray(), columns=tokens)
print('"""\n', X_train[0], '\n"""')
X_train_as_pd[['cat', 'dog', 'it', 'itself', 'the', 'there', 'you']].head()

(3007, 43366)
['inexact', 'inexcusable', 'inexorable', 'inexorably', 'inexpensive', 'inexpensively', 'inexperienced', 'inf', 'infact', 'infalability', 'infalibility', 'infallable', 'infallibale', 'infallibility', 'infallible', 'infallibly', 'infamous', 'infamy', 'infancy', 'infant']
"""
 From: mlee@eng.sdsu.edu (Mike Lee)
Subject: MPEG for x-windows MONO needed.
Organization: San Diego State University Computing Services
Lines: 4
NNTP-Posting-Host: eng.sdsu.edu
X-Newsreader: TIN [version 1.1 PL9]

Hello, and thank you for reading this request.  I have a Mpeg viewer for x-windows and it did not run because I was running it on a monochrome monitor.  I need the mono-driver for mpeg_play.   

Please post the location of the file or better yet, e-mail me at mlee@eng.sdsu.edu.

 
"""


,cat,dog,it,itself,the,there,you
0,0,0,2,0,3,0,1
1,0,0,0,0,2,0,3
2,0,0,7,0,7,1,2
3,0,0,3,0,15,1,10
4,0,0,0,0,2,0,4


## Step2 (v1): predict based on cos similarity

We test a k-NN based on cosinus similarity. To do so, we use scikit-learn implementation of k-NN, which use L$_2$ distance between examples, and we feed it with normalized examples (the normalization is example per example). Therefore, the model look for the nearest neighbors given
$$\left\|\frac{x}{\|x\|}-\frac{y}{\|y\|}\right\|
= \left(\frac{\|x\|}{\|x\|}\right)^2 + \left(\frac{\|y\|}{\|y\|}\right)^2-2\left<\frac{x}{\|x\|},\frac{y}{\|y\|}\right>
= 1 + 1 - 2 \cos(x,y), $$
which is equivalent to maximizing the cosinus between examples.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import Normalizer

model = Pipeline([
    ('norm', Normalizer()),
    ('clf', KNeighborsClassifier(n_neighbors=3)),
])
model.fit(X_train_counts, y_train)

n_prints = 3
for (sample, pred) in zip(X_test[:n_prints], model.predict(X_test_counts[:n_prints])):
  print(f'"""\nDocument : {sample[len(sample)*3//4:len(sample)*3//4+200]}\n"""  Prediction : {twenty.target_names[pred]}')
print("\n")
#to heavy to compute
print("Training accuracy: ", model.score(X_train_counts, y_train))
print("Test accuracy: ", model.score(X_test_counts, y_test))

"""
Document : y (World magazine) warned us all that it was very dangerous,
probably to stop curious children from experimenting with it.  Mind you, this
was 10 years ago, at least.  (And boy, does that say somethin
"""  Prediction : sci.med
"""
Document : e
that it is, is every bit as absurd as to assert "it is not raining" on
a rainy day.  I take this to be a candidate for an objective value, and it
it is a necessary condition for objective morality t
"""  Prediction : alt.atheism
"""
Document :    June 1992, from the Institute for Creation Research).  This is one
     of the most offensive articles they've ever published--but at least
     it argues *against* a deathbed conversion.  There's 
"""  Prediction : soc.religion.christian


Training accuracy:  0.8875956102427669
Test accuracy:  0.7393617021276596


Sur 100 documents du jeu de test, le modèle K-NN en classe correctement environ 74% dans la bonne catégorie.

## Step2 (v2): predict based on a trained (simple) model


In [ ]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB().fit(X_train_counts, y_train)

n_prints = 3
for (sample, pred) in zip(X_test[:n_prints], model.predict(X_test_counts[:n_prints])):
  print(f'"""\nDocument : {sample[len(sample)*3//4:len(sample)*3//4+200]}\n"""  Prediction : {twenty.target_names[pred]}')
print("\n")
print("Training accuracy: ", model.score(X_train_counts, y_train))
print("Test accuracy: ", model.score(X_test_counts, y_test))

"""
Document : y (World magazine) warned us all that it was very dangerous,
probably to stop curious children from experimenting with it.  Mind you, this
was 10 years ago, at least.  (And boy, does that say somethin
"""  Prediction : sci.med
"""
Document : e
that it is, is every bit as absurd as to assert "it is not raining" on
a rainy day.  I take this to be a candidate for an objective value, and it
it is a necessary condition for objective morality t
"""  Prediction : alt.atheism
"""
Document :    June 1992, from the Institute for Creation Research).  This is one
     of the most offensive articles they've ever published--but at least
     it argues *against* a deathbed conversion.  There's 
"""  Prediction : alt.atheism


Training accuracy:  0.9946790821416694
Test accuracy:  0.9720744680851063


Les textes du jeu de test sont majoritairement classés dans la bonne catégorie 97%.

## Complete process

In [ ]:
model1 = Pipeline([
    ('vect', CountVectorizer()),
    ('norm', Normalizer()),
    ('clf', KNeighborsClassifier(n_neighbors=3)),
])

model2 = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB()),
])

for model in [model1, model2]:
  print(f'===\n{model.named_steps}\n===')
  model.fit(X_train, y_train)
  n_prints = 3
  for (sample, pred) in zip(X_test[:n_prints], model.predict(X_test[:n_prints])):
    print(f'"""\nDocument : {sample[len(sample)*3//4:len(sample)*3//4+200]}\n"""  Prediction : {twenty.target_names[pred]}')
  print("Test accuracy: ", model.score(X_test, y_test))
  print('-'*60)

===
{'vect': CountVectorizer(), 'norm': Normalizer(), 'clf': KNeighborsClassifier(n_neighbors=3)}
===
"""
Document : y (World magazine) warned us all that it was very dangerous,
probably to stop curious children from experimenting with it.  Mind you, this
was 10 years ago, at least.  (And boy, does that say somethin
"""  Prediction : sci.med
"""
Document : e
that it is, is every bit as absurd as to assert "it is not raining" on
a rainy day.  I take this to be a candidate for an objective value, and it
it is a necessary condition for objective morality t
"""  Prediction : alt.atheism
"""
Document :    June 1992, from the Institute for Creation Research).  This is one
     of the most offensive articles they've ever published--but at least
     it argues *against* a deathbed conversion.  There's 
"""  Prediction : soc.religion.christian
Test accuracy:  0.7393617021276596
------------------------------------------------------------
===
{'vect': CountVectorizer(), 'clf': MultinomialNB()}
=

# Variations

## TO DO: Implement and test the following variations

## tf-idf vectorization

TF-IDF (Term Frequency – Inverse Document Frequency) pèse les mots selon leur importance dans chaque document, tout en pénalisant les mots trop fréquents dans l’ensemble des documents (souvent moins discriminants).

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

model_tfidf1 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', KNeighborsClassifier(n_neighbors=3)),
])

model_tfidf2 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB()),
])

for model in [model_tfidf1, model_tfidf2]:
  print(f'===\n{model.named_steps}\n===')
  model.fit(X_train, y_train)
  n_prints = 2
  for (sample, pred) in zip(X_test[:n_prints], model.predict(X_test[:n_prints])):
    print(f'"""\nDocument : {sample[len(sample)*3//4:len(sample)*3//4+200]}\n"""  Prediction : {twenty.target_names[pred]}')
  print("Test accuracy: ", model.score(X_test, y_test))
  print('-'*60)

===
{'tfidf': TfidfVectorizer(), 'clf': KNeighborsClassifier(n_neighbors=3)}
===
"""
Document : y (World magazine) warned us all that it was very dangerous,
probably to stop curious children from experimenting with it.  Mind you, this
was 10 years ago, at least.  (And boy, does that say somethin
"""  Prediction : sci.med
"""
Document : e
that it is, is every bit as absurd as to assert "it is not raining" on
a rainy day.  I take this to be a candidate for an objective value, and it
it is a necessary condition for objective morality t
"""  Prediction : alt.atheism
Test accuracy:  0.8949468085106383
------------------------------------------------------------
===
{'tfidf': TfidfVectorizer(), 'clf': MultinomialNB()}
===
"""
Document : y (World magazine) warned us all that it was very dangerous,
probably to stop curious children from experimenting with it.  Mind you, this
was 10 years ago, at least.  (And boy, does that say somethin
"""  Prediction : sci.med
"""
Document : e
that it is, is 

## Hyper-parameters of the vectorizer
Test (separatly) the impact of
* lowercase=False
* stop_words='english'
* binary=True
* ngram_range = (1,2) et (1,3)

In [ ]:
#  lowercase=False
#  Permet de conserver la casse, ce qui peut être pertinent si des mots en majuscule ont un sens différent (acronymes, noms propres, etc.).

model_tfidf1 = Pipeline([
    ('tfidf', TfidfVectorizer(lowercase=False)),
    ('clf', KNeighborsClassifier(n_neighbors=3)),
])

model_tfidf2 = Pipeline([
    ('tfidf', TfidfVectorizer(lowercase=False)),
    ('clf', MultinomialNB()),
])

for model in [model_tfidf1, model_tfidf2]:
  print(f'===\n{model.named_steps}\n===')
  model.fit(X_train, y_train)
  n_prints = 2
  for (sample, pred) in zip(X_test[:n_prints], model.predict(X_test[:n_prints])):
    print(f'"""\nDocument : {sample[len(sample)*3//4:len(sample)*3//4+200]}\n"""  Prediction : {twenty.target_names[pred]}')
  print("Test accuracy: ", model.score(X_test, y_test))
  print('-'*60)

===
{'tfidf': TfidfVectorizer(lowercase=False), 'clf': KNeighborsClassifier(n_neighbors=3)}
===
"""
Document : y (World magazine) warned us all that it was very dangerous,
probably to stop curious children from experimenting with it.  Mind you, this
was 10 years ago, at least.  (And boy, does that say somethin
"""  Prediction : sci.med
"""
Document : e
that it is, is every bit as absurd as to assert "it is not raining" on
a rainy day.  I take this to be a candidate for an objective value, and it
it is a necessary condition for objective morality t
"""  Prediction : alt.atheism
Test accuracy:  0.8962765957446809
------------------------------------------------------------
===
{'tfidf': TfidfVectorizer(lowercase=False), 'clf': MultinomialNB()}
===
"""
Document : y (World magazine) warned us all that it was very dangerous,
probably to stop curious children from experimenting with it.  Mind you, this
was 10 years ago, at least.  (And boy, does that say somethin
"""  Prediction : sci.med
""

In [ ]:
# stop_words='english'
# Permet de retirer automatiquement la liste de mots vides (stopwords) en anglais (ex.: “the”, “and”, “of”, etc.), qui n’apportent généralement pas de valeur discriminante.

model_tfidf1 = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', KNeighborsClassifier(n_neighbors=3)),
])

model_tfidf2 = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', MultinomialNB()),
])

for model in [model_tfidf1, model_tfidf2]:
  print(f'===\n{model.named_steps}\n===')
  model.fit(X_train, y_train)

  print("Test accuracy: ", model.score(X_test, y_test))
  print('-'*60)

===
{'tfidf': TfidfVectorizer(stop_words='english'), 'clf': KNeighborsClassifier(n_neighbors=3)}
===
Test accuracy:  0.9162234042553191
------------------------------------------------------------
===
{'tfidf': TfidfVectorizer(stop_words='english'), 'clf': MultinomialNB()}
===
Test accuracy:  0.949468085106383
------------------------------------------------------------


In [ ]:
# binary=True

model_tfidf1 = Pipeline([
    ('tfidf', TfidfVectorizer(binary=True)),
    ('clf', KNeighborsClassifier(n_neighbors=3)),
])

model_tfidf2 = Pipeline([
    ('tfidf', TfidfVectorizer(binary=True)),
    ('clf', MultinomialNB()),
])

for model in [model_tfidf1, model_tfidf2]:
  print(f'===\n{model.named_steps}\n===')
  model.fit(X_train, y_train)

  print("Test accuracy: ", model.score(X_test, y_test))
  print('-'*60)

===
{'tfidf': TfidfVectorizer(binary=True), 'clf': KNeighborsClassifier(n_neighbors=3)}
===
Test accuracy:  0.9375
------------------------------------------------------------
===
{'tfidf': TfidfVectorizer(binary=True), 'clf': MultinomialNB()}
===
Test accuracy:  0.910904255319149
------------------------------------------------------------


In [ ]:
# ngram_range=(1,2)

model_tfidf1 = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2))),
    ('clf', KNeighborsClassifier(n_neighbors=3)),
])

model_tfidf2 = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2))),
    ('clf', MultinomialNB()),
])

for model in [model_tfidf1, model_tfidf2]:
  print(f'===\n{model.named_steps}\n===')
  model.fit(X_train, y_train)

  print("Test accuracy: ", model.score(X_test, y_test))
  print('-'*60)

===
{'tfidf': TfidfVectorizer(ngram_range=(1, 2)), 'clf': KNeighborsClassifier(n_neighbors=3)}
===
Test accuracy:  0.9002659574468085
------------------------------------------------------------
===
{'tfidf': TfidfVectorizer(ngram_range=(1, 2)), 'clf': MultinomialNB()}
===
Test accuracy:  0.8909574468085106
------------------------------------------------------------


## Stemming (with a pretrained tokenizer)

Take inspiration from [this page](https://jonathansoma.com/lede/algorithms-2017/classes/more-text-analysis/counting-and-stemming/).

Le stemming consiste à réduire un mot à sa racine (“fishing”, “fishes”,“fished”→ “fish”).
La lemmatisation est plus “linguistique” (“better” → “good”), mais nécessite un lemmatiseur spécifique (spaCy, NLTK, etc.).

In [ ]:
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Téléchargement des ressources nécessaires pour le stemming
nltk.download('punkt')
nltk.download('punkt_tab')

# Initialisation du stemmer
stemmer = PorterStemmer()

# Fonction pour tokenizer et appliquer le stemming
def my_tokenizer(text):
    """
    Tokenise le texte en mots, puis applique le stemming à chaque mot.
    """
    tokens = word_tokenize(text)  # Tokenizer de NLTK
    stemmed_tokens = [stemmer.stem(token) for token in tokens]  # Appliquer le stemming
    return ' '.join(stemmed_tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# Instanciation de vectorizer (TF-IDF ou Count) en lui passant ce tokenizer
# vectorizer = TfidfVectorizer(tokenizer=my_tokenizer, stop_words='english')

In [ ]:
# Transformation des données d’entraînement et de test en vecteurs
X_train_stemmed = [my_tokenizer(doc) for doc in X_train]
X_test_stemmed = [my_tokenizer(doc) for doc in X_test]

In [ ]:
# Vectorisation avec CountVectorizer (ou TF-IDF si vous préférez)
vectorizer = CountVectorizer()
X_train_stemmed_counts = vectorizer.fit_transform(X_train_stemmed)
X_test_stemmed_counts = vectorizer.transform(X_test_stemmed)

In [ ]:
# Entraînement du modèle (Naive Bayes, SVM, etc.) sur ces vecteurs
model_stemmed = MultinomialNB()
model_stemmed.fit(X_train_stemmed_counts, y_train)

MultinomialNB()

In [ ]:
accuracy_stemmed = model_stemmed.score(X_test_stemmed_counts, y_test)
print(f"Précision après Stemming (CountVectorizer + Naive Bayes) : {accuracy_stemmed:.4f}")

Précision après Stemming (CountVectorizer + Naive Bayes) : 0.9668


## Another ML algorithm

In [ ]:
from sklearn.linear_model import LogisticRegression

model_lr = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', LogisticRegression(max_iter=1000)),
])
model_lr.fit(X_train, y_train)
print("LR Test accuracy:", model_lr.score(X_test, y_test))

LR Test accuracy: 0.9521276595744681
